# San Antonio Hotel Simulations
8760 rows per simulation<br>
Floors for residential and commercial

## Preprocess

1. Generate zone with facade glazing systems in COMFEN.

2. Retrieve IDFs for each scenario from the COMFEN5 Results folder. Have to do each of the 3 sections of the building as its own sqlite file and each project by orientation separately. Then IDF files are copied after each simulation. These files will be used in the Jupyter Notebook.

3. Convert from E+ 8.1 to 9.5 using the .lst format.
4. Change the reporting in the IDF files to "All" to get the HTML summary table and the inch-pound units for verification.
      
    * OutputControl:Table:Style,
    * All,                     !- Column Separator
    * InchPound;               !- Unit Conversion
5. Rerun batch sims in EnergyPlus v9.5.
6. Add "Scenario", "Elevation, and "GlazingSystem" to the results CSV files.

In [1]:
# Dependencies
import pandas as pd
import os
import numpy as np
from pathlib import Path
from datetime import datetime

## Energy Conversions

As per E* Documentation<br>

Joules --> kBtu (multiply by .00000094781712)<br>
Joules --> kWh (multiply by .000000277777778) or (divide by 3600000)<br>
Joules --> therms (multiply by .0000000094781712<br>

<div>
    <img src="static/images/elect_conv.png" width="400"/>
     <img src="static/images/gas_conv.png" width="400"/>
</div

## Energy Costs and Emission Factors

http://seeatcalc.gastechnology.org/ResidentialBuildings.aspx

In [2]:
# Costs (elec $/kWh, gas $/therm), change values per city
comgas_cost = 0.60
comelec_cost = 0.0806

In [3]:
# Source Energy Conversion Factors (same for res or comm), change values per city
gas_source = 1.09
elec_source = 2.53

In [4]:
# Carbon equivalent factors (same for res or comm), change values per city

#Electricity (lb/MMBtu) 323.34 (lb/kbtu).32334 ####
#Natural Gas (Building Used, lb/MMBtu) 145.66 (lb/kBtu .14566) ####

gas_co2 = 0.14566
elec_co2 = 0.42103

In [5]:
# Zone size for EUI and cost/sf

sf = 189.6

In [6]:
# Lighting energy reduction, assumption that LEDs are being used. 
# Use this multiplier if reductions are not in the COMFEN model

lighting_redux = 1

In [7]:
# # For peak demand calculations (.0002931 kW = 1 Btu).

# kw = 0.0002931

---

## Combine CSV Files in Directory Commercial Electric Heating

In [8]:
# Create variable for files in directory
files = [f for f in os.listdir("data/sat/") if f.endswith(".csv")]

# files

In [9]:
# https://stackoverflow.com/questions/63886787/how-to-create-a-dataframe-from-multiple-csv-files
# Initialize and empty dataframe
all_ComElec = pd.DataFrame()

# Iterate through files and contents, then concatenate the data into the dataframe
for file in files:
    df = pd.read_csv("data/sat/" + file)
    all_ComElec = pd.concat([all_ComElec, df])
    
# all_ComElec

# 394200  rows, 44 columns

In [10]:
# list(all_ComElec.columns.values)

In [11]:
# Get wanted columns
all_ComElec1 = all_ComElec[["Scenario",
               "Date/Time",
                "Elevation",
               "GlazingSystem",
              "SYS-1 HEATING COIL:Heating Coil Electricity Energy [J](Hourly)",
              "SYS-1:Air System NaturalGas Energy [J](Hourly)",
              "SYS-1:Air System Fan Electricity Energy [J](Hourly)",
              "SYS-1:Air System DX Cooling Coil Electricity Energy [J](Hourly) ",
             "COMFENZONE:Zone Lights Electricity Energy [J](Hourly)"]]

# all_ComElec1

In [12]:
all_ComElec1 = all_ComElec1.dropna().reset_index(drop=True)

#all_ComElec1

In [13]:
# # Export grouped file to csv
# all_ComElec1.to_csv("Scraper_Output/sat_all.csv", header=True, index=True)

-----

## Get the Wanted Data (columns)

In [14]:
# Rename Columns
all_ComElec2 = all_ComElec1.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "SYS-1:Air System Fan Electricity Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "SYS-1:Air System NaturalGas Energy [J](Hourly)":"HeatingGasEnergy[J](Hourly)",
                            "SYS-1 HEATING COIL:Heating Coil Electricity Energy [J](Hourly)":"HeatingElecEnergy[J](Hourly)",
                            "SYS-1:Air System DX Cooling Coil Electricity Energy [J](Hourly) ":"CoolingEnergy[J](Hourly)",
                            "COMFENZONE:Zone Lights Electricity Energy [J](Hourly)":"LightingEnergy[J](Hourly)"
                             })

#all_ComElec2

-----

## Get Annual Energy Data
Convert units from Joules to kBtu<br>

In [15]:
# Copy dataframe to modify and leave original df intact
get_annual_ComElec = all_ComElec2.copy(deep=True)

#get_annual_ComElec

In [16]:
# Convert Joules to KBtu in dataframe
get_annual_ComElec["FanEnergy(kBtu)"] = get_annual_ComElec["FanEnergy[J](Hourly)"] * .00000094781712
get_annual_ComElec["HeatingElecEnergy(kBtu)"] = get_annual_ComElec["HeatingElecEnergy[J](Hourly)"] * .00000094781712
get_annual_ComElec["HeatingGasEnergy(kBtu)"] = get_annual_ComElec["HeatingGasEnergy[J](Hourly)"] * .00000094781712
get_annual_ComElec["CoolingEnergy(kBtu)"] = get_annual_ComElec["CoolingEnergy[J](Hourly)"] * .00000094781712

# Reduce lighting energy, assuming LEDs are being used
get_annual_ComElec["LightingEnergy(kBtu)"] = (get_annual_ComElec["LightingEnergy[J](Hourly)"] * .00000094781712) * lighting_redux

#get_annual_ComElec

In [17]:
# Drop columns
get_annual_ComElec.drop(columns = ["FanEnergy[J](Hourly)", "HeatingElecEnergy[J](Hourly)", "HeatingGasEnergy[J](Hourly)", "CoolingEnergy[J](Hourly)", "LightingEnergy[J](Hourly)"], axis=1, inplace=True)

#get_annual_ComElec

In [18]:
# Add energy totals
totalComEnergy = get_annual_ComElec["FanEnergy(kBtu)"] + get_annual_ComElec["HeatingElecEnergy(kBtu)"] + get_annual_ComElec["LightingEnergy(kBtu)"] + get_annual_ComElec["CoolingEnergy(kBtu)"]
        
get_annual_ComElec["AnnualEnergy(kBtu)"] = totalComEnergy

# get_annual_ComElec

-----

## Get Energy per sf 
Add EUI (site) column<br>


In [19]:
# Get data per sf (energy/sf)
get_annual_ComElec["FanEnergy(kBtu/sf)"] = (get_annual_ComElec["FanEnergy(kBtu)"]) / sf
get_annual_ComElec["HeatingElecEnergy(kBtu/sf)"] = (get_annual_ComElec["HeatingElecEnergy(kBtu)"]) / sf
get_annual_ComElec["HeatingGasEnergy(kBtu/sf)"] = (get_annual_ComElec["HeatingGasEnergy(kBtu)"]) / sf
get_annual_ComElec["CoolingEnergy(kBtu/sf)"] = (get_annual_ComElec["CoolingEnergy(kBtu)"]) / sf
get_annual_ComElec["LightingEnergy(kBtu/sf)"] = (get_annual_ComElec["LightingEnergy(kBtu)"]) / sf

In [20]:
# Establish EUI (energy/sf)
get_annual_ComElec["EUI(kBtu/sf)"] = (get_annual_ComElec["FanEnergy(kBtu)"] +
                                    get_annual_ComElec["HeatingElecEnergy(kBtu)"] +
                                    get_annual_ComElec["HeatingGasEnergy(kBtu)"] +
                                    get_annual_ComElec["LightingEnergy(kBtu)"] +  
                                    get_annual_ComElec["CoolingEnergy(kBtu)"]) / sf
                                                       
#get_annual_ComElec

-----

## Get Carbon per sf 
Add carbon/sf column

In [21]:
# Do carbon calculation using variable (lb/sf)
get_annual_ComElec["CO2_Electric(lb/sf)"] = ((get_annual_ComElec["FanEnergy(kBtu)"] +
                                    get_annual_ComElec["HeatingElecEnergy(kBtu)"] +
                                    get_annual_ComElec["LightingEnergy(kBtu)"] +
                                    get_annual_ComElec["CoolingEnergy(kBtu)"]) * elec_co2) / sf
                                                       
# get_annual_ComElec

In [22]:
get_annual_ComElec["CO2_Gas(lb/sf)"] = ((get_annual_ComElec["HeatingGasEnergy(kBtu)"]) * gas_co2) / sf
                                                       
# get_annual_ComElec

## Get Annual Cost Data
Convert units from Joules to kWh and therms to use cost multiplier<br>
Add cost multiplier, variables defined above

In [23]:
# Get annual costs
get_annual_costs_ComElec = all_ComElec2.copy(deep=True)

In [24]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_annual_costs_ComElec["FanCost($)"] = (get_annual_costs_ComElec["FanEnergy[J](Hourly)"] * .000000277777778) * comelec_cost
get_annual_costs_ComElec["HeatingElecCost($)"] = ((get_annual_costs_ComElec["HeatingElecEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * comgas_cost
get_annual_costs_ComElec["HeatingGasCost($)"] = ((get_annual_costs_ComElec["HeatingGasEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * comgas_cost
get_annual_costs_ComElec["CoolingCost($)"] = (get_annual_costs_ComElec["CoolingEnergy[J](Hourly)"] * .000000277777778) * comelec_cost

# Reduce lighting energy, assuming LEDs are being used
get_annual_costs_ComElec["LightingCost($)"] = (get_annual_costs_ComElec["LightingEnergy[J](Hourly)"] * .00000094781712) * lighting_redux

#get_annual_costs

In [25]:
# Drop columns
get_annual_costs_ComElec.drop(columns = ["FanEnergy[J](Hourly)", "HeatingElecEnergy[J](Hourly)", "HeatingGasEnergy[J](Hourly)", "CoolingEnergy[J](Hourly)", "LightingEnergy[J](Hourly)"], axis=1, inplace=True)

#get_annual_costs_ComElec

In [26]:
# Add cost totals
totalComCost = get_annual_costs_ComElec["FanCost($)"] + get_annual_costs_ComElec["HeatingElecCost($)"] + get_annual_costs_ComElec["HeatingGasCost($)"]  + get_annual_costs_ComElec["CoolingCost($)"] + get_annual_costs_ComElec["LightingCost($)"]
        
get_annual_costs_ComElec["AnnualCost($)"] = totalComCost

# get_annual_costs_ComElec

-----

## Get Annual Cost per sf Data
Add costs/sf columns<br>


In [27]:
# Get data per sf ($/sf)
get_annual_costs_ComElec["FanCost($/sf)"] = (get_annual_costs_ComElec["FanCost($)"]) / sf
get_annual_costs_ComElec["HeatingElecCost($/sf)"] = (get_annual_costs_ComElec["HeatingElecCost($)"]) / sf
get_annual_costs_ComElec["HeatingGasCost($/sf)"] = (get_annual_costs_ComElec["HeatingGasCost($)"]) / sf
get_annual_costs_ComElec["CoolingCost($/sf)"] = (get_annual_costs_ComElec["CoolingCost($)"]) / sf
get_annual_costs_ComElec["LightingCost($/sf)"] = (get_annual_costs_ComElec["LightingCost($)"]) / sf

# get_annual_costs_ComElec

In [28]:
# Establish costs ($/sf)
get_annual_costs_ComElec["AnnualCost($/sf)"] = (get_annual_costs_ComElec["FanCost($)"] +
                                    get_annual_costs_ComElec["HeatingElecCost($)"] +
                                    get_annual_costs_ComElec["HeatingGasCost($)"] +  
                                    get_annual_costs_ComElec["CoolingCost($)"] +  
                                    get_annual_costs_ComElec["LightingCost($)"]) / sf
                                                       
# get_annual_costs_ComElec

---

## Combine Annual Energy and Cost Data
Combine two dfs<br>
Export combine file

In [29]:
# # Combine data for energy and cost
allAnnualDataComElec = pd.merge(get_annual_ComElec, get_annual_costs_ComElec)

# allAnnualDataComElec

In [30]:
# list(allAnnualDataComElec.columns.values)

---
## Group and Export Annual Data
Group dataframe by scenario<br>
Export to file

In [31]:
# Group by scenario
annualDataCom = allAnnualDataComElec.groupby(["Scenario", "Elevation", "GlazingSystem"], as_index=True)

# annualDataCom.sum().round(2)

In [32]:
annualDataComFinal = annualDataCom.sum().round(2)

# annualDataComFinal

In [33]:
# Export grouped file to csv
annualDataComFinal.to_csv("Scraper_Output/sat_annual_com.csv", header=True, index=True)

---

## Establish dataframes for each condition to determine delta from existing

Create dataset with just existing condition.<br> 
Created a dataset with clear panel. <br>
Create a dataset with lowe panel.<br>
Keep the units in kBtus to be able to eventually combine heating and cooling <br>
Export to CSV files


In [34]:
# Define the scenario for the existing condition.
baselineAnnualDataComElec = allAnnualDataComElec[allAnnualDataComElec["GlazingSystem"] == "Exist"]

# baselineAnnualDataComElec

In [35]:
# Rename columns to indicate baseline to prepare for merge"

allBaselineAnnualDataComElec = baselineAnnualDataComElec.rename(columns={'Scenario':'B_Scenario',
 'Date_Time':'B_Date_Time',
 'Elevation':'B_Elevation',
 'GlazingSystem':'B_GlazingSystem',
 'FanEnergy(kBtu)':'B_FanEnergy(kBtu)',
 'HeatingElecEnergy(kBtu)':'B_HeatingElecEnergy(kBtu)',
 'HeatingGasEnergy(kBtu)':'B_HeatingGasEnergy(kBtu)',
 'CoolingEnergy(kBtu)':'B_CoolingEnergy(kBtu)',
 'LightingEnergy(kBtu)':'B_LightingEnergy(kBtu)',
 'AnnualEnergy(kBtu)':'B_AnnualEnergy(kBtu)',
 'FanEnergy(kBtu/sf)':'B_FanEnergy(kBtu/sf)',
 'HeatingElecEnergy(kBtu/sf)':'B_HeatingElecEnergy(kBtu/sf)',
 'HeatingGasEnergy(kBtu/sf)':'B_HeatingGasEnergy(kBtu/sf)',
 'CoolingEnergy(kBtu/sf)':'B_CoolingEnergy(kBtu/sf)',
 'LightingEnergy(kBtu/sf)':'B_LightingEnergy(kBtu/sf)',
 'EUI(kBtu/sf)':'B_EUI(kBtu/sf)',
 'CO2_Electric(lb/sf)':'B_CO2_Electric(lb/sf)',
 'CO2_Gas(lb/sf)':'B_CO2_Gas(lb/sf)',
 'FanCost($)':'B_FanCost($)',
 'HeatingElecCost($)':'B_HeatingElecCost($)',
 'HeatingGasCost($)':'B_HeatingGasCost($)',
 'CoolingCost($)':'B_CoolingCost($)',
 'LightingCost($)':'B_LightingCost($)',
 'AnnualCost($)':'B_AnnualCost($)',
 'FanCost($/sf)':'B_FanCost($/sf)',
 'HeatingElecCost($/sf)':'B_HeatingElecCost($/sf)',
 'HeatingGasCost($/sf)':'B_HeatingGasCost($/sf)',
 'CoolingCost($/sf)':'B_CoolingCost($/sf)',
 'LightingCost($/sf)':'B_LightingCost($/sf)',
 'AnnualCost($/sf)':'B_AnnualCost($/sf)'
                            })

# allBaselineAnnualDataComElec

In [36]:
# Reset index
allBaselineAnnualDataComElec.reset_index(inplace=True, drop=True)

# allBaselineAnnualDataComElec

In [37]:
# # Define the scenario for the baselines.

clearAnnualDataComElec = allAnnualDataComElec[allAnnualDataComElec["GlazingSystem"] == "Clear"]

# clearAnnualDataComElec

In [38]:
# Rename columns to indicate baseline to prepare for merge"

allClearAnnualDataComElec = clearAnnualDataComElec.rename(columns={'Scenario':'C_Scenario',
 'Date_Time':'C_Date_Time',
 'Elevation':'C_Elevation',
 'GlazingSystem':'C_GlazingSystem',
 'FanEnergy(kBtu)':'C_FanEnergy(kBtu)',
 'HeatingElecEnergy(kBtu)':'C_HeatingElecEnergy(kBtu)',
 'HeatingGasEnergy(kBtu)':'C_HeatingGasEnergy(kBtu)',
 'CoolingEnergy(kBtu)':'C_CoolingEnergy(kBtu)',
 'LightingEnergy(kBtu)':'C_LightingEnergy(kBtu)',
 'AnnualEnergy(kBtu)':'C_AnnualEnergy(kBtu)',
 'FanEnergy(kBtu/sf)':'C_FanEnergy(kBtu/sf)',
 'HeatingElecEnergy(kBtu/sf)':'C_HeatingElecEnergy(kBtu/sf)',
 'HeatingGasEnergy(kBtu/sf)':'C_HeatingGasEnergy(kBtu/sf)',
 'CoolingEnergy(kBtu/sf)':'C_CoolingEnergy(kBtu/sf)',
 'LightingEnergy(kBtu/sf)':'C_LightingEnergy(kBtu/sf)',
 'EUI(kBtu/sf)':'C_EUI(kBtu/sf)',
 'CO2_Electric(lb/sf)':'C_CO2_Electric(lb/sf)',
 'CO2_Gas(lb/sf)':'C_CO2_Gas(lb/sf)',
 'FanCost($)':'C_FanCost($)',
 'HeatingElecCost($)':'C_HeatingElecCost($)',
 'HeatingGasCost($)':'C_HeatingGasCost($)',
 'CoolingCost($)':'C_CoolingCost($)',
 'LightingCost($)':'C_LightingCost($)',
 'AnnualCost($)':'C_AnnualCost($)',
 'FanCost($/sf)':'C_FanCost($/sf)',
 'HeatingElecCost($/sf)':'C_HeatingElecCost($/sf)',
 'HeatingGasCost($/sf)':'C_HeatingGasCost($/sf)',
 'CoolingCost($/sf)':'C_CoolingCost($/sf)',
 'LightingCost($/sf)':'C_LightingCost($/sf)',
 'AnnualCost($/sf)':'C_AnnualCost($/sf)'
                            })

# allClearAnnualDataComElec

In [39]:
# Reset index
allClearAnnualDataComElec.reset_index(inplace=True, drop=True)

# allClearAnnualDataComElec

In [40]:
# Define the scenario for the baselines.

loweAnnualDataComElec = allAnnualDataComElec[allAnnualDataComElec["GlazingSystem"] == "Lowe"]

# loweAnnualDataComElec

In [41]:
# Rename columns to indicate baseline to prepare for merge"

allLoweAnnualDataComElec = loweAnnualDataComElec.rename(columns={'Scenario':'L_Scenario',
 'Date_Time':'L_Date_Time',
 'Elevation':'L_Elevation',
 'GlazingSystem':'L_GlazingSystem',
 'FanEnergy(kBtu)':'L_FanEnergy(kBtu)',
 'HeatingElecEnergy(kBtu)':'L_HeatingElecEnergy(kBtu)',
 'HeatingGasEnergy(kBtu)':'L_HeatingGasEnergy(kBtu)',
 'CoolingEnergy(kBtu)':'L_CoolingEnergy(kBtu)',
 'LightingEnergy(kBtu)':'L_LightingEnergy(kBtu)',
 'AnnualEnergy(kBtu)':'L_AnnualEnergy(kBtu)',
 'FanEnergy(kBtu/sf)':'L_FanEnergy(kBtu/sf)',
 'HeatingElecEnergy(kBtu/sf)':'L_HeatingElecEnergy(kBtu/sf)',
 'HeatingGasEnergy(kBtu/sf)':'L_HeatingGasEnergy(kBtu/sf)',
 'CoolingEnergy(kBtu/sf)':'L_CoolingEnergy(kBtu/sf)',
 'LightingEnergy(kBtu/sf)':'L_LightingEnergy(kBtu/sf)',
 'EUI(kBtu/sf)':'L_EUI(kBtu/sf)',
 'CO2_Electric(lb/sf)':'L_CO2_Electric(lb/sf)',
 'CO2_Gas(lb/sf)':'L_CO2_Gas(lb/sf)',
 'FanCost($)':'L_FanCost($)',
 'HeatingElecCost($)':'L_HeatingElecCost($)',
 'HeatingGasCost($)':'L_HeatingGasCost($)',
 'CoolingCost($)':'L_CoolingCost($)',
 'LightingCost($)':'L_LightingCost($)',
 'AnnualCost($)':'L_AnnualCost($)',
 'FanCost($/sf)':'L_FanCost($/sf)',
 'HeatingElecCost($/sf)':'L_HeatingElecCost($/sf)',
 'HeatingGasCost($/sf)':'L_HeatingGasCost($/sf)',
 'CoolingCost($/sf)':'L_CoolingCost($/sf)',
 'LightingCost($/sf)':'L_LightingCost($/sf)',
 'AnnualCost($/sf)':'L_AnnualCost($/sf)'
                            })

# allLoweAnnualDataComElec

In [42]:
#Reset index
allLoweAnnualDataComElec.reset_index(inplace=True, drop=True)

# allLoweAnnualDataComElec

In [43]:
# # Export individual CSV files for each condition

# allBaselineAnnualDataComElec.to_csv("Scraper_Output/sat_exist_all_annual_com.csv", header=True, index=True)
# allClearAnnualDataComElec.to_csv("Scraper_Output/sat_clear_all_annual_com.csv", header=True, index=True)
# allLoweAnnualDataComElec.to_csv("Scraper_Output/sat_lowe_all_annual_com.csv", header=True, index=True)

---
## Merge the 3 dataframes to perform delta from existing calculation


In [44]:
# merge existing and clear
MergedAnnualData = pd.merge(allBaselineAnnualDataComElec, allClearAnnualDataComElec, left_index=True, right_index=True)

# merge the above with lowe
AllMergedAnnualData = pd.merge(MergedAnnualData, allLoweAnnualDataComElec, left_index=True, right_index=True)

# AllMergedAnnualData

In [45]:
# list(AllMergedAnnualData.columns.values)

In [46]:
# Deltas of exist - clear

AllMergedAnnualData["DeltaClear_FanEnergy(kBtu/sf)"] = AllMergedAnnualData["B_FanEnergy(kBtu/sf)"] - AllMergedAnnualData["C_FanEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaClear_HeatingElecEnergy(kBtu/sf)"] = AllMergedAnnualData["B_HeatingElecEnergy(kBtu/sf)"] - AllMergedAnnualData["C_HeatingElecEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaClear_HeatingGasEnergy(kBtu/sf)"] = AllMergedAnnualData["B_HeatingGasEnergy(kBtu/sf)"] - AllMergedAnnualData["C_HeatingGasEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaClear_CoolingEnergy(kBtu/sf)"] = AllMergedAnnualData["B_CoolingEnergy(kBtu/sf)"] - AllMergedAnnualData["C_CoolingEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaClear_LightingEnergy(kBtu/sf)"] = AllMergedAnnualData["B_LightingEnergy(kBtu/sf)"] - AllMergedAnnualData["C_LightingEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaClear_EUI(kBtu/sf)"] = AllMergedAnnualData["B_EUI(kBtu/sf)"] - AllMergedAnnualData["C_EUI(kBtu/sf)"]
AllMergedAnnualData["DeltaClear_CO2_Electric(lb/sf)"] = AllMergedAnnualData["B_CO2_Electric(lb/sf)"] - AllMergedAnnualData["C_CO2_Electric(lb/sf)"]
AllMergedAnnualData["DeltaClear_CO2_Gas(lb/sf)"] = AllMergedAnnualData["B_CO2_Gas(lb/sf)"] - AllMergedAnnualData["C_CO2_Gas(lb/sf)"]
AllMergedAnnualData["DeltaClear_FanCost($)"] = AllMergedAnnualData["B_FanCost($)"] - AllMergedAnnualData["C_FanCost($)"]
AllMergedAnnualData["DeltaClear_HeatingElecCost($)"] = AllMergedAnnualData["B_HeatingElecCost($)"] - AllMergedAnnualData["C_HeatingElecCost($)"]
AllMergedAnnualData["DeltaClear_HeatingGasCost($)"] = AllMergedAnnualData["B_HeatingGasCost($)"] - AllMergedAnnualData["C_HeatingGasCost($)"]
AllMergedAnnualData["DeltaClear_CoolingCost($)"] = AllMergedAnnualData["B_CoolingCost($)"] - AllMergedAnnualData["C_CoolingCost($)"]
AllMergedAnnualData["DeltaClear_LightingCost($)"] = AllMergedAnnualData["B_LightingCost($)"] - AllMergedAnnualData["C_LightingCost($)"]
AllMergedAnnualData["DeltaClear_AnnualCost($)"] = AllMergedAnnualData["B_AnnualCost($)"] - AllMergedAnnualData["C_AnnualCost($)"]
AllMergedAnnualData["DeltaClear_FanCost($/sf)"] = AllMergedAnnualData["B_FanCost($/sf)"] - AllMergedAnnualData["C_FanCost($/sf)"]
AllMergedAnnualData["DeltaClear_HeatingElecCost($/sf)"] = AllMergedAnnualData["B_HeatingElecCost($/sf)"] - AllMergedAnnualData["C_HeatingElecCost($/sf)"]
AllMergedAnnualData["DeltaClear_HeatingGasCost($/sf)"] = AllMergedAnnualData["B_HeatingGasCost($/sf)"] - AllMergedAnnualData["C_HeatingGasCost($/sf)"]
AllMergedAnnualData["DeltaClear_CoolingCost($/sf)"] = AllMergedAnnualData["B_CoolingCost($/sf)"] - AllMergedAnnualData["C_CoolingCost($/sf)"]
AllMergedAnnualData["DeltaClear_LightingCost($/sf)"] = AllMergedAnnualData["B_LightingCost($/sf)"] - AllMergedAnnualData["C_LightingCost($/sf)"]
AllMergedAnnualData["DeltaClear_AnnualCost($/sf)"] = AllMergedAnnualData["B_AnnualCost($/sf)"] - AllMergedAnnualData["C_AnnualCost($/sf)"]


In [47]:
# list(AllMergedAnnualData.columns.values)

In [48]:
# Deltas of exist - lowe

AllMergedAnnualData["DeltaLowe_FanEnergy(kBtu/sf)"] = AllMergedAnnualData["B_FanEnergy(kBtu/sf)"] - AllMergedAnnualData["L_FanEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaLowe_HeatingElecEnergy(kBtu/sf)"] = AllMergedAnnualData["B_HeatingElecEnergy(kBtu/sf)"] - AllMergedAnnualData["L_HeatingElecEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaLowe_HeatingGasEnergy(kBtu/sf)"] = AllMergedAnnualData["B_HeatingGasEnergy(kBtu/sf)"] - AllMergedAnnualData["L_HeatingGasEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaLowe_CoolingEnergy(kBtu/sf)"] = AllMergedAnnualData["B_CoolingEnergy(kBtu/sf)"] - AllMergedAnnualData["L_CoolingEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaLowe_LightingEnergy(kBtu/sf)"] = AllMergedAnnualData["B_LightingEnergy(kBtu/sf)"] - AllMergedAnnualData["L_LightingEnergy(kBtu/sf)"]
AllMergedAnnualData["DeltaLowe_EUI(kBtu/sf)"] = AllMergedAnnualData["B_EUI(kBtu/sf)"] - AllMergedAnnualData["L_EUI(kBtu/sf)"]
AllMergedAnnualData["DeltaLowe_CO2_Electric(lb/sf)"] = AllMergedAnnualData["B_CO2_Electric(lb/sf)"] - AllMergedAnnualData["L_CO2_Electric(lb/sf)"]
AllMergedAnnualData["DeltaLowe_CO2_Gas(lb/sf)"] = AllMergedAnnualData["B_CO2_Gas(lb/sf)"] - AllMergedAnnualData["L_CO2_Gas(lb/sf)"]
AllMergedAnnualData["DeltaLowe_FanCost($)"] = AllMergedAnnualData["B_FanCost($)"] - AllMergedAnnualData["L_FanCost($)"]
AllMergedAnnualData["DeltaLowe_HeatingElecCost($)"] = AllMergedAnnualData["B_HeatingElecCost($)"] - AllMergedAnnualData["L_HeatingElecCost($)"]
AllMergedAnnualData["DeltaLowe_HeatingGasCost($)"] = AllMergedAnnualData["B_HeatingGasCost($)"] - AllMergedAnnualData["L_HeatingGasCost($)"]
AllMergedAnnualData["DeltaLowe_CoolingCost($)"] = AllMergedAnnualData["B_CoolingCost($)"] - AllMergedAnnualData["L_CoolingCost($)"]
AllMergedAnnualData["DeltaLowe_LightingCost($)"] = AllMergedAnnualData["B_LightingCost($)"] - AllMergedAnnualData["L_LightingCost($)"]
AllMergedAnnualData["DeltaLowe_AnnualCost($)"] = AllMergedAnnualData["B_AnnualCost($)"] - AllMergedAnnualData["L_AnnualCost($)"]
AllMergedAnnualData["DeltaLowe_FanCost($/sf)"] = AllMergedAnnualData["B_FanCost($/sf)"] - AllMergedAnnualData["L_FanCost($/sf)"]
AllMergedAnnualData["DeltaLowe_HeatingElecCost($/sf)"] = AllMergedAnnualData["B_HeatingElecCost($/sf)"] - AllMergedAnnualData["L_HeatingElecCost($/sf)"]
AllMergedAnnualData["DeltaLowe_HeatingGasCost($/sf)"] = AllMergedAnnualData["B_HeatingGasCost($/sf)"] - AllMergedAnnualData["L_HeatingGasCost($/sf)"]
AllMergedAnnualData["DeltaLowe_CoolingCost($/sf)"] = AllMergedAnnualData["B_CoolingCost($/sf)"] - AllMergedAnnualData["L_CoolingCost($/sf)"]
AllMergedAnnualData["DeltaLowe_LightingCost($/sf)"] = AllMergedAnnualData["B_LightingCost($/sf)"] - AllMergedAnnualData["L_LightingCost($/sf)"]
AllMergedAnnualData["DeltaLowe_AnnualCost($/sf)"] = AllMergedAnnualData["B_AnnualCost($/sf)"] - AllMergedAnnualData["L_AnnualCost($/sf)"]


In [49]:
# list(AllMergedAnnualData.columns.values)

---
## Group and Export Annual Delta Data
Group dataframe by scenario<br>
Export to file

In [50]:
# Group by scenario
AnnualDataDelta = AllMergedAnnualData.groupby(["B_Scenario", "B_Elevation", "B_GlazingSystem"], as_index=True)

# AnnualDataDelta.sum().round(2)

In [51]:
AnnualDataDelta = AnnualDataDelta.sum().round(2)

In [52]:
# list(AnnualDataDelta.sum().round(2).columns.values)

In [53]:
# Export grouped file to csv
AnnualDataDelta.to_csv("Scraper_Output/sat_deltas_annual_com.csv", header=True, index=True)

## Find Peak Demand
Slice data to find peak demand for each scenario<br>
Export to csv

In [54]:
# Copy dataframe to modify and leave original df intact
get_peak_ComElec = all_ComElec2.copy(deep=True)

# get_peak_ComElec

In [55]:
# Add energy totals
# divide by the number of seconds in an hour (3600)
totalComPeak = (get_peak_ComElec["FanEnergy[J](Hourly)"] + get_peak_ComElec["HeatingElecEnergy[J](Hourly)"] + get_peak_ComElec["HeatingGasEnergy[J](Hourly)"] + get_peak_ComElec["CoolingEnergy[J](Hourly)"] + get_peak_ComElec["LightingEnergy[J](Hourly)"])
        
get_peak_ComElec["PeakEnergy[J](Hourly)"] = totalComPeak

get_peak_ComElec["PeakEnergy(w)"] = (get_peak_ComElec["PeakEnergy[J](Hourly)"] / 3600)

get_peak_ComElec["PeakEnergy(w/sf)"] = (get_peak_ComElec["PeakEnergy(w)"] / sf)

# get_peak_ComElec

In [56]:
# Get scenario and find the max
A1Eexist = get_peak_ComElec[get_peak_ComElec["Scenario"] == "A1Eexist"]
A1Eclear = get_peak_ComElec[get_peak_ComElec["Scenario"] == "A1Eclear"]
A1Elowe = get_peak_ComElec[get_peak_ComElec["Scenario"] == "A1Elowe"]
A1Nexist = get_peak_ComElec[get_peak_ComElec["Scenario"] == "A1Nexist"]
A1Nclear = get_peak_ComElec[get_peak_ComElec["Scenario"] == "A1Nclear"]
A1Nlowe = get_peak_ComElec[get_peak_ComElec["Scenario"] == "A1Nlowe"]
A1Wexist = get_peak_ComElec[get_peak_ComElec["Scenario"] == "A1Wexist"]
A1Wclear = get_peak_ComElec[get_peak_ComElec["Scenario"] == "A1Wclear"]
A1Wlowe = get_peak_ComElec[get_peak_ComElec["Scenario"] == "A1Wlowe"]
A2Sexist = get_peak_ComElec[get_peak_ComElec["Scenario"] == "A2Sexist"]
A2Sclear = get_peak_ComElec[get_peak_ComElec["Scenario"] == "A2Sclear"]
A2Slowe = get_peak_ComElec[get_peak_ComElec["Scenario"] == "A2Slowe"]
B1Eexist = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B1Eexist"]
B1Eclear = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B1Eclear"]
B1Elowe = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B1Elowe"]
B1Nexist = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B1Nexist"]
B1Nclear = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B1Nclear"]
B1Nlowe = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B1Nlowe"]
B1Wexist = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B1Wexist"]
B1Wclear = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B1Wclear"]
B1Wlowe = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B1Wlowe"]
B2Eexist = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B2Eexist"]
B2Eclear = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B2Eclear"]
B2Elowe = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B2Elowe"]
B3Sexist = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B3Sexist"]
B3Sclear = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B3Sclear"]
B3Slowe = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B3Slowe"]
B3Wexist = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B3Wexist"]
B3Wclear = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B3Wclear"]
B3Wlowe = get_peak_ComElec[get_peak_ComElec["Scenario"] == "B3Wlowe"]
C1Eexist = get_peak_ComElec[get_peak_ComElec["Scenario"] == "C1Eexist"]
C1Eclear = get_peak_ComElec[get_peak_ComElec["Scenario"] == "C1Eclear"]
C1Elowe = get_peak_ComElec[get_peak_ComElec["Scenario"] == "C1Elowe"]
C1Nexist = get_peak_ComElec[get_peak_ComElec["Scenario"] == "C1Nexist"]
C1Nclear = get_peak_ComElec[get_peak_ComElec["Scenario"] == "C1Nclear"]
C1Nlowe = get_peak_ComElec[get_peak_ComElec["Scenario"] == "C1Nlowe"]
C1Sexist = get_peak_ComElec[get_peak_ComElec["Scenario"] == "C1Sexist"]
C1Sclear = get_peak_ComElec[get_peak_ComElec["Scenario"] == "C1Sclear"]
C1Slowe = get_peak_ComElec[get_peak_ComElec["Scenario"] == "C1Slowe"]
C2Wexist = get_peak_ComElec[get_peak_ComElec["Scenario"] == "C2Wexist"]
C2Wclear = get_peak_ComElec[get_peak_ComElec["Scenario"] == "C2Wclear"]
C2Wlowe = get_peak_ComElec[get_peak_ComElec["Scenario"] == "C2Wlowe"]
C3Wexist = get_peak_ComElec[get_peak_ComElec["Scenario"] == "C3Wexist"]
C3Wclear = get_peak_ComElec[get_peak_ComElec["Scenario"] == "C3Wclear"]
C3Wlowe = get_peak_ComElec[get_peak_ComElec["Scenario"] == "C3Wlowe"]

In [57]:
A1EexistPeak = (A1Eexist[A1Eexist["PeakEnergy[J](Hourly)"] == A1Eexist["PeakEnergy[J](Hourly)"].max()])
A1EclearPeak = (A1Eclear[A1Eclear["PeakEnergy[J](Hourly)"] == A1Eclear["PeakEnergy[J](Hourly)"].max()])
A1ElowePeak = (A1Elowe[A1Elowe["PeakEnergy[J](Hourly)"] == A1Elowe["PeakEnergy[J](Hourly)"].max()])
A1NexistPeak = (A1Nexist[A1Nexist["PeakEnergy[J](Hourly)"] == A1Nexist["PeakEnergy[J](Hourly)"].max()])
A1NclearPeak = (A1Nclear[A1Nclear["PeakEnergy[J](Hourly)"] == A1Nclear["PeakEnergy[J](Hourly)"].max()])
A1NlowePeak = (A1Nlowe[A1Nlowe["PeakEnergy[J](Hourly)"] == A1Nlowe["PeakEnergy[J](Hourly)"].max()])
A1WexistPeak = (A1Wexist[A1Wexist["PeakEnergy[J](Hourly)"] == A1Wexist["PeakEnergy[J](Hourly)"].max()])
A1WclearPeak = (A1Wclear[A1Wclear["PeakEnergy[J](Hourly)"] == A1Wclear["PeakEnergy[J](Hourly)"].max()])
A1WlowePeak = (A1Wlowe[A1Wlowe["PeakEnergy[J](Hourly)"] == A1Wlowe["PeakEnergy[J](Hourly)"].max()])
A2SexisttPeak = (A2Sexist[A2Sexist["PeakEnergy[J](Hourly)"] == A2Sexist["PeakEnergy[J](Hourly)"].max()])
A2SclearPeak = (A2Sclear[A2Sclear["PeakEnergy[J](Hourly)"] == A2Sclear["PeakEnergy[J](Hourly)"].max()])
A2SlowePeak = (A2Slowe[A2Slowe["PeakEnergy[J](Hourly)"] == A2Slowe["PeakEnergy[J](Hourly)"].max()])
B1EexistPeak = (B1Eexist[B1Eexist["PeakEnergy[J](Hourly)"] == B1Eexist["PeakEnergy[J](Hourly)"].max()])
B1EclearPeak = (B1Eclear[B1Eclear["PeakEnergy[J](Hourly)"] == B1Eclear["PeakEnergy[J](Hourly)"].max()])
B1ElowePeak = (B1Elowe[B1Elowe["PeakEnergy[J](Hourly)"] == B1Elowe["PeakEnergy[J](Hourly)"].max()])
B1NexistPeak = (B1Nexist[B1Nexist["PeakEnergy[J](Hourly)"] == B1Nexist["PeakEnergy[J](Hourly)"].max()])
B1NclearPeak = (B1Nclear[B1Nclear["PeakEnergy[J](Hourly)"] == B1Nclear["PeakEnergy[J](Hourly)"].max()])
B1NlowePeak = (B1Nlowe[B1Nlowe["PeakEnergy[J](Hourly)"] == B1Nlowe["PeakEnergy[J](Hourly)"].max()])
B1WexistPeak = (B1Wexist[B1Wexist["PeakEnergy[J](Hourly)"] == B1Wexist["PeakEnergy[J](Hourly)"].max()])
B1WclearPeak = (B1Wclear[B1Wclear["PeakEnergy[J](Hourly)"] == B1Wclear["PeakEnergy[J](Hourly)"].max()])
B1WlowePeak = (B1Wlowe[B1Wlowe["PeakEnergy[J](Hourly)"] == B1Wlowe["PeakEnergy[J](Hourly)"].max()])
B2EexistPeak = (B2Eexist[B2Eexist["PeakEnergy[J](Hourly)"] == B2Eexist["PeakEnergy[J](Hourly)"].max()])
B2EclearPeak = (B2Eclear[B2Eclear["PeakEnergy[J](Hourly)"] == B2Eclear["PeakEnergy[J](Hourly)"].max()])
B2ElowePeak = (B2Elowe[B2Elowe["PeakEnergy[J](Hourly)"] == B2Elowe["PeakEnergy[J](Hourly)"].max()])
B3SexistPeak = (B3Sexist[B3Sexist["PeakEnergy[J](Hourly)"] == B3Sexist["PeakEnergy[J](Hourly)"].max()])
B3SclearPeak = (B3Sclear[B3Sclear["PeakEnergy[J](Hourly)"] == B3Sclear["PeakEnergy[J](Hourly)"].max()])
B3SlowePeak = (B3Slowe[B3Slowe["PeakEnergy[J](Hourly)"] == B3Slowe["PeakEnergy[J](Hourly)"].max()])
B3WexistPeak = (B3Wexist[B3Wexist["PeakEnergy[J](Hourly)"] == B3Wexist["PeakEnergy[J](Hourly)"].max()])
B3WclearPeak = (B3Wclear[B3Wclear["PeakEnergy[J](Hourly)"] == B3Wclear["PeakEnergy[J](Hourly)"].max()])
B3WlowePeak = (B3Wlowe[B3Wlowe["PeakEnergy[J](Hourly)"] == B3Wlowe["PeakEnergy[J](Hourly)"].max()])
C1EexistPeak = (C1Eexist[C1Eexist["PeakEnergy[J](Hourly)"] == C1Eexist["PeakEnergy[J](Hourly)"].max()])
C1EclearPeak = (C1Eclear[C1Eclear["PeakEnergy[J](Hourly)"] == C1Eclear["PeakEnergy[J](Hourly)"].max()])
C1ElowePeak = (C1Elowe[C1Elowe["PeakEnergy[J](Hourly)"] == C1Elowe["PeakEnergy[J](Hourly)"].max()])
C1NexistPeak = (C1Nexist[C1Nexist["PeakEnergy[J](Hourly)"] == C1Nexist["PeakEnergy[J](Hourly)"].max()])
C1NclearPeak = (C1Nclear[C1Nclear["PeakEnergy[J](Hourly)"] == C1Nclear["PeakEnergy[J](Hourly)"].max()])
C1NlowePeak = (C1Nlowe[C1Nlowe["PeakEnergy[J](Hourly)"] == C1Nlowe["PeakEnergy[J](Hourly)"].max()])
C1SexistPeak = (C1Sexist[C1Sexist["PeakEnergy[J](Hourly)"] == C1Sexist["PeakEnergy[J](Hourly)"].max()])
C1SclearPeak = (C1Sclear[C1Sclear["PeakEnergy[J](Hourly)"] == C1Sclear["PeakEnergy[J](Hourly)"].max()])
C1SlowePeak = (C1Slowe[C1Slowe["PeakEnergy[J](Hourly)"] == C1Slowe["PeakEnergy[J](Hourly)"].max()])
C2WexistPeak = (C2Wexist[C2Wexist["PeakEnergy[J](Hourly)"] == C2Wexist["PeakEnergy[J](Hourly)"].max()])
C2WclearPeak = (C2Wclear[C2Wclear["PeakEnergy[J](Hourly)"] == C2Wclear["PeakEnergy[J](Hourly)"].max()])
C2WlowePeak = (C2Wlowe[C2Wlowe["PeakEnergy[J](Hourly)"] == C2Wlowe["PeakEnergy[J](Hourly)"].max()])
C3WexistPeak = (C3Wexist[C3Wexist["PeakEnergy[J](Hourly)"] == C3Wexist["PeakEnergy[J](Hourly)"].max()])
C3WclearPeak = (C3Wclear[C3Wclear["PeakEnergy[J](Hourly)"] == C3Wclear["PeakEnergy[J](Hourly)"].max()])
C3WlowePeak = (C3Wlowe[C3Wlowe["PeakEnergy[J](Hourly)"] == C3Wlowe["PeakEnergy[J](Hourly)"].max()])

In [58]:
PeakDemandElec = A1EexistPeak.append([A1EclearPeak,
A1ElowePeak,
A1NexistPeak,
A1NclearPeak,
A1NlowePeak,
A1WexistPeak,
A1WclearPeak,
A1WlowePeak,
A2SexisttPeak,
A2SclearPeak,
A2SlowePeak,
B1EexistPeak,
B1EclearPeak,
B1ElowePeak,
B1NexistPeak,
B1NclearPeak,
B1NlowePeak,
B1WexistPeak,
B1WclearPeak,
B1WlowePeak,
B2EexistPeak,
B2EclearPeak,
B2ElowePeak,
B3SexistPeak,
B3SclearPeak,
B3SlowePeak,
B3WexistPeak,
B3WclearPeak,
B3WlowePeak,
C1EexistPeak,
C1EclearPeak,
C1ElowePeak,
C1NexistPeak,
C1NclearPeak,
C1NlowePeak,
C1SexistPeak,
C1SclearPeak,
C1SlowePeak,
C2WexistPeak,
C2WclearPeak,
C2WlowePeak,
C3WexistPeak,
C3WclearPeak,
C3WlowePeak])

# PeakDemandElec

In [59]:
# Export dataframe  to csv
PeakDemandElec.to_csv("Scraper_Output/sat_peak_com.csv", header=True, index=True)

---

## Establish dataframes for each condition to determine delta from existing

Create dataset with just existing condition.<br> 
Created a dataset with clear panel. <br>
Create a dataset with lowe panel.<br>
Export to CSV files


In [60]:
# Define the scenario for the existing condition.
baselinePeakDemandElec = PeakDemandElec[PeakDemandElec["GlazingSystem"] == "Exist"]

# baselinePeakDemandElec

In [61]:
# list(baselinePeakDemandElec.columns.values)

In [62]:
# Rename columns to indicate baseline to prepare for merge"

allbaselinePeakDemandElec = baselinePeakDemandElec.rename(columns={'Scenario':'B_Scenario',
 'Date_Time':'B_Date_Time',
 'Elevation':'Elevation',
 'GlazingSystem':'GlazingSystem',
 'HeatingElecEnergy[J](Hourly)':'B_HeatingElecEnergy[J](Hourly)',
 'HeatingGasEnergy[J](Hourly)':'B_HeatingGasEnergy[J](Hourly)',
 'FanEnergy[J](Hourly)':'B_FanEnergy[J](Hourly)',
 'CoolingEnergy[J](Hourly)':'B_CoolingEnergy[J](Hourly)',
 'LightingEnergy[J](Hourly)':'B_LightingEnergy[J](Hourly)',
 'PeakEnergy[J](Hourly)':'B_PeakEnergy[J](Hourly)',
 'PeakEnergy(w)':'B_PeakEnergy(w)',
 'PeakEnergy(w/sf)':'B_PeakEnergy(w/sf)'
                            })

# allbaselinePeakDemandElec

In [63]:
#Reset index
allbaselinePeakDemandElec.reset_index(inplace=True, drop=True)

# allbaselinePeakDemandElec

In [64]:
# Define the scenario for the clear panel condition.
clearPeakDemandElec = PeakDemandElec[PeakDemandElec["GlazingSystem"] == "Clear"]

# clearPeakDemandElec

In [65]:
# Rename columns to indicate baseline to prepare for merge"

allClearPeakDemandElec = clearPeakDemandElec.rename(columns={'Scenario':'C_Scenario',
 'Date_Time':'C_Date_Time',
 'Elevation':'C_Elevation',
 'GlazingSystem':'C_GlazingSystem',
 'HeatingElecEnergy[J](Hourly)':'C_HeatingElecEnergy[J](Hourly)',
 'HeatingGasEnergy[J](Hourly)':'C_HeatingGasEnergy[J](Hourly)',
 'FanEnergy[J](Hourly)':'C_FanEnergy[J](Hourly)',
 'CoolingEnergy[J](Hourly)':'C_CoolingEnergy[J](Hourly)',
 'LightingEnergy[J](Hourly)':'C_LightingEnergy[J](Hourly)',
 'PeakEnergy[J](Hourly)':'C_PeakEnergy[J](Hourly)',
 'PeakEnergy(w)':'C_PeakEnergy(w)',
 'PeakEnergy(w/sf)':'C_PeakEnergy(w/sf)'
                            })

# allClearPeakDemandElec

In [66]:
#Reset index
allClearPeakDemandElec.reset_index(inplace=True, drop=True)

# allClearPeakDemandElec

In [67]:
# Define the scenario for the lowe panel condition.
lowePeakDemandElec = PeakDemandElec[PeakDemandElec["GlazingSystem"] == "Lowe"]

# lowePeakDemandElec

In [68]:
# Rename columns to indicate baseline to prepare for merge"

allLowePeakDemandElec = lowePeakDemandElec.rename(columns={'Scenario':'L_Scenario',
 'Date_Time':'L_Date_Time',
 'Elevation':'L_Elevation',
 'GlazingSystem':'L_GlazingSystem',
 'HeatingElecEnergy[J](Hourly)':'L_HeatingElecEnergy[J](Hourly)',
 'HeatingGasEnergy[J](Hourly)':'L_HeatingGasEnergy[J](Hourly)',
 'FanEnergy[J](Hourly)':'L_FanEnergy[J](Hourly)',
 'CoolingEnergy[J](Hourly)':'L_CoolingEnergy[J](Hourly)',
 'LightingEnergy[J](Hourly)':'L_LightingEnergy[J](Hourly)',
 'PeakEnergy[J](Hourly)':'L_PeakEnergy[J](Hourly)',
 'PeakEnergy(w)':'L_PeakEnergy(w)',
 'PeakEnergy(w/sf)':'L_PeakEnergy(w/sf)'
                            })

# allLowePeakDemandElec

In [69]:
# Reset index
allLowePeakDemandElec.reset_index(inplace=True, drop=True)

# allLowePeakDemandElec

---
## Merge the 3 dataframes to perform delta peak from existing calculation


In [70]:
# merge existing and clear
MergedAnnualPeakData = pd.merge(allbaselinePeakDemandElec, allClearPeakDemandElec, left_index=True, right_index=True)

MergedAnnualPeakData

#merge the above with lowe
AllMergedAnnualPeakData = pd.merge(MergedAnnualPeakData, allLowePeakDemandElec, left_index=True, right_index=True)

# AllMergedAnnualPeakData

In [71]:
# list(AllMergedAnnualPeakData.columns.values)

In [72]:
#Deltas of exist - clear

AllMergedAnnualPeakData["DeltaClear_PeakEnergy(w)"] = AllMergedAnnualPeakData["B_PeakEnergy(w)"] - AllMergedAnnualPeakData["C_PeakEnergy(w)"]
AllMergedAnnualPeakData["DeltaClear_PeakEnergy(w/sf)"] = AllMergedAnnualPeakData["B_PeakEnergy(w/sf)"] - AllMergedAnnualPeakData["C_PeakEnergy(w/sf)"]


In [73]:
# Deltas of exist - lowe

AllMergedAnnualPeakData["DeltaLowe_PeakEnergy(w)"] = AllMergedAnnualPeakData["B_PeakEnergy(w)"] - AllMergedAnnualPeakData["L_PeakEnergy(w)"]
AllMergedAnnualPeakData["DeltaLowe_PeakEnergy(w/sf)"] = AllMergedAnnualPeakData["B_PeakEnergy(w/sf)"] - AllMergedAnnualPeakData["L_PeakEnergy(w/sf)"]

# AllMergedAnnualPeakData

In [74]:
# list(AllMergedAnnualPeakData.columns.values)

In [75]:
# Export grouped file to csv
AllMergedAnnualPeakData.to_csv("Scraper_Output/sat_deltas_peak_com.csv", header=True, index=True)